# HMS 3.x to 4.x Conversion Workflow

This notebook demonstrates how to convert an HMS 3.x project to run with HMS 4.x, with comprehensive QAQC documentation following the **LLM Forward Approach**.

## LLM Forward QAQC Requirements

1. **Modeling Change Log** - Document all modifications made during conversion
2. **Results Comparison** - Compare outputs between original (3.x) and converted (4.x) versions
3. **Visual Verification** - Generate hydrograph comparison figures
4. **Difference Flagging** - Identify any discrepancies for engineer review
5. **Comprehensive Coverage** - Compare ALL hydrological elements (subbasins, reaches, junctions)

## Background

HEC-HMS version 3.x projects may encounter compatibility issues when opened in HMS 4.x:

| Issue | HMS 3.x | HMS 4.x | Impact on Results |
|-------|---------|---------|-------------------|
| Met model basin reference | Pattern matching | Exact name required | None (structural) |
| Cross-section table case | Case-insensitive | Case-sensitive | None (structural) |
| Muskingum Cunge params | Optional | Required | **Potential** (hydraulic) |
| Jython syntax | Python 2 | Python 3 | None (execution) |

This example uses the Clear Creek (A100-00-00) project from the HCFCD M3 Model archives.

In [ ]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

**Note**: This notebook **requires M3 data download** and **HMS 3.x and 4.x installations** for full execution.

## Setup

In [ ]:
from pathlib import Path
from datetime import datetime
import shutil
import json

import pandas as pd
import numpy as np

from hms_commander import HmsM3Model, HmsJython, __version__

# For plotting
try:
    import matplotlib.pyplot as plt
    HAS_MATPLOTLIB = True
except ImportError:
    HAS_MATPLOTLIB = False
    print("matplotlib not installed - figures will be skipped")

print(f"hms-commander v{__version__}")

## Initialize Modeling Change Log

All modifications will be documented in this log for QAQC purposes.

In [ ]:
# Initialize modeling change log
# Note: source_version and target_version will be updated after detecting installed versions
modeling_log = {
    "project_name": "A100-00-00 (Clear Creek)",
    "conversion_date": datetime.now().isoformat(),
    "source_version": "HMS 3.x (to be detected)",
    "target_version": "HMS 4.x (to be detected)",
    "engineer": "[To be filled by reviewing engineer]",
    "changes": [],
    "warnings": [],
    "results_comparison": {}
}

def log_change(category, description, file_affected, old_value, new_value, impact="None"):
    """Add a change to the modeling log."""
    change = {
        "category": category,
        "description": description,
        "file": file_affected,
        "old_value": old_value,
        "new_value": new_value,
        "expected_impact": impact,
        "timestamp": datetime.now().isoformat()
    }
    modeling_log["changes"].append(change)
    print(f"[LOG] {category}: {description}")
    print(f"      File: {file_affected}")
    print(f"      Old: {old_value}")
    print(f"      New: {new_value}")
    print(f"      Expected Impact: {impact}")
    print()

def log_warning(message):
    """Add a warning to the modeling log."""
    modeling_log["warnings"].append({
        "message": message,
        "timestamp": datetime.now().isoformat()
    })
    print(f"[WARNING] {message}")

def update_log_versions(source_version, target_version):
    """Update the modeling log with detected versions."""
    modeling_log["source_version"] = source_version
    modeling_log["target_version"] = target_version

print("Modeling change log initialized.")
print(f"Project: {modeling_log['project_name']}")
print("Source and target versions will be detected automatically.")

## Step 1: Extract HMS 3.x Project

In [ ]:
# Define paths
m3_cache = Path.cwd() / 'hms_example_projects' / 'Upgrade_M3_to_4x'
m3_cache.mkdir(parents=True, exist_ok=True)

# Extract Clear Creek project (A100-00-00) from Model A archive
print("Extracting Clear Creek (A100-00-00) from M3 archive...")
project_path = HmsM3Model.extract_project(
    model_id="A",
    unit_id="A100-00-00",
    output_path=m3_cache,
    overwrite=True
)
print(f"Project extracted to: {project_path}")

# Dynamically detect installed HMS versions
from hms_commander import HmsExamples

print("\nDetecting installed HMS versions...")
installed_versions = HmsExamples.detect_installed_versions()

# Find HMS 3.x and 4.x versions
hms_33_path = None
hms_411_path = None

for version, path in installed_versions.items():
    version_parts = version.split('.')
    major = int(version_parts[0])
    
    if major == 3:
        # Prefer 3.3 specifically, but use any 3.x
        if version == "3.3" or hms_33_path is None:
            hms_33_path = path
            print(f"Found HMS 3.x: {version} at {path}")
    elif major == 4:
        # Prefer 4.11 specifically, but use any 4.x
        if version == "4.11" or hms_411_path is None:
            hms_411_path = path
            print(f"Found HMS 4.x: {version} at {path}")

# Update modeling log with detected versions
source_ver = f"HMS {hms_33_path.name}" if hms_33_path else "HMS 3.x (not installed)"
target_ver = f"HMS {hms_411_path.name}" if hms_411_path else "HMS 4.x (not installed)"
update_log_versions(source_ver, target_ver)

# Check available versions
print(f"\nHMS 3.x path: {hms_33_path}")
print(f"HMS 3.x available: {hms_33_path is not None and hms_33_path.exists()}")
print(f"HMS 4.x path: {hms_411_path}")
print(f"HMS 4.x available: {hms_411_path is not None and hms_411_path.exists()}")
print(f"\nModeling log versions: {source_ver} -> {target_ver}")

if not hms_33_path or not hms_411_path:
    print("\n[WARNING] This notebook requires both HMS 3.x and HMS 4.x to be installed.")
    print(f"Installed versions: {list(installed_versions.keys())}")

## Step 2: Run Original Project with HMS 3.3 (Baseline)

In [ ]:
# Run with HMS 3.x (native version) - This is our BASELINE
run_name = "1%(100YR)RUN"

if hms_33_path and hms_33_path.exists():
    script = HmsJython.generate_compute_script(
        project_path=project_path,
        run_name=run_name,
        python2_compatible=True  # HMS 3.x uses Jython 2.5
    )
    
    print(f"Executing BASELINE run with HMS {hms_33_path.name}...")
    result_33 = HmsJython.execute_script(
        script_content=script,
        hms_exe_path=hms_33_path,
        working_dir=project_path,
        timeout=300
    )
    
    print(f"\nBaseline Success: {result_33[0]}")
    if "Computation completed" in result_33[1]:
        print(f"HMS {hms_33_path.name} baseline run completed successfully.")
        modeling_log["baseline_run"] = {
            "version": f"HMS {hms_33_path.name}",
            "run_name": run_name,
            "success": True,
            "dss_file": str(project_path / "A1000000.dss")
        }
else:
    print("HMS 3.x not installed - cannot establish baseline")
    log_warning("No HMS 3.x baseline available - comparison will be limited")

## Step 3: Create Copy for HMS 4.11 Conversion

In [ ]:
# Create a copy for HMS 4.11 testing
project_411 = project_path.parent / f"{project_path.name}_HMS411"

if project_411.exists():
    shutil.rmtree(project_411)

shutil.copytree(project_path, project_411)
print(f"Created copy for HMS 4.11 testing: {project_411}")

log_change(
    category="Project Setup",
    description="Created project copy for HMS 4.11 conversion",
    file_affected=str(project_411),
    old_value="N/A",
    new_value=f"Copy of {project_path.name}",
    impact="None - working on copy preserves original"
)

## Step 4: Apply Conversion Fixes with Full Logging

In [ ]:
# Fix 1: Met file basin model reference
print("=" * 60)
print("FIX 1: Met Model Basin Reference")
print("=" * 60)

met_file = project_411 / "1__24HR.met"
if met_file.exists():
    content = met_file.read_text(encoding='utf-8', errors='ignore')
    
    old_basin_ref = "Use Basin Model: A100"
    new_basin_ref = "Use Basin Model: A100_1PCT"
    
    if old_basin_ref in content:
        content = content.replace(old_basin_ref, new_basin_ref)
        met_file.write_text(content, encoding='utf-8')
        
        log_change(
            category="Met Model",
            description="Updated basin model reference from pattern to exact name",
            file_affected="1__24HR.met",
            old_value=old_basin_ref,
            new_value=new_basin_ref,
            impact="None - structural change only, same basin model used"
        )
    else:
        print("Basin reference already updated or not found.")
else:
    print(f"Met file not found: {met_file}")

In [ ]:
# Fix 2 & 3: Muskingum Cunge parameters
print("=" * 60)
print("FIX 2 & 3: Muskingum Cunge Cross-Section and Parameters")
print("=" * 60)

# Document the Muskingum Cunge parameter changes in detail
muskingum_changes = {
    "reach_name": "A1009999_2147_R",
    "routing_method": "Muskingum Cunge",
    "channel_type": "8-point",
    "parameters_preserved": {
        "Length": "17107 ft",
        "Energy Slope": "0.0005",
        "Manning's n (all)": "0.04"
    },
    "parameters_modified": {
        "Cross Section Name": {"old": "Table 5", "new": "TABLE 5", "reason": "Case sensitivity fix"},
        "Manning's n naming": {"old": "Left/Main/Right Overbank", "new": "Mannings n, Left/Right Mannings n", "reason": "HMS 4.x naming convention"}
    },
    "parameters_added": {
        "Index Parameter Type": "Index Celerity",
        "Index Celerity": "5",
        "Space-Time Method": "Automatic DX and DT",
        "Maximum Depth Iterations": "20",
        "Maximum Route Step Iterations": "30"
    }
}

print(f"\nMuskingum Cunge Parameter Documentation:")
print(f"  Reach: {muskingum_changes['reach_name']}")
print(f"  Method: {muskingum_changes['routing_method']}")
print(f"  Channel: {muskingum_changes['channel_type']}")
print("\n  PRESERVED Parameters (unchanged):")
for param, value in muskingum_changes['parameters_preserved'].items():
    print(f"    {param}: {value}")
print("\n  MODIFIED Parameters:")
for param, details in muskingum_changes['parameters_modified'].items():
    print(f"    {param}: {details['old']} -> {details['new']}")
    print(f"      Reason: {details['reason']}")
print("\n  ADDED Parameters (HMS 4.x requirements):")
for param, value in muskingum_changes['parameters_added'].items():
    print(f"    {param}: {value}")

# Store for later reference
modeling_log["muskingum_cunge_details"] = muskingum_changes

In [ ]:
# Apply the Muskingum Cunge fix to basin files
old_muskingum = '''     Route: Muskingum Cunge
     Channel: 8-point
     Length: 17107
     Energy Slope: 0.0005
     Left Overbank Mannings n: 0.04
     Main Channel Mannings n: 0.04
     Right Overbank Mannings n: 0.04
     Cross Section Name: Table 5
     Use Variable Time Step: No
     Channel Loss: None'''

new_muskingum = '''     Route: Muskingum Cunge
     Channel: 8-point
     Length: 17107
     Energy Slope: 0.0005
     Mannings n: 0.04
     Left Mannings n: 0.04
     Right Mannings n: 0.04
     Cross Section Name: TABLE 5
     Initial Variable: Combined Inflow
     Index Parameter Type: Index Celerity
     Index Celerity: 5
     Space-Time Method: Automatic DX and DT
     Maximum Depth Iterations: 20
     Maximum Route Step Iterations: 30
     Channel Loss: None'''

files_modified = []
for basin_file in project_411.glob("*.basin"):
    content = basin_file.read_text(encoding='utf-8', errors='ignore')
    
    if old_muskingum in content:
        new_content = content.replace(old_muskingum, new_muskingum)
        basin_file.write_text(new_content, encoding='utf-8')
        files_modified.append(basin_file.name)

if files_modified:
    log_change(
        category="Reach Routing",
        description="Updated Muskingum Cunge parameters for HMS 4.x compatibility",
        file_affected=", ".join(files_modified),
        old_value="HMS 3.x Muskingum Cunge (Table 5, no Index params)",
        new_value="HMS 4.x Muskingum Cunge (TABLE 5, Index Celerity=5)",
        impact="POTENTIAL - Index Celerity value affects wave speed calculation. ENGINEER REVIEW RECOMMENDED."
    )
    
    log_warning(
        "Muskingum Cunge Index Celerity set to 5 fps. This affects wave propagation speed. "
        "Original HMS 3.x model may have used different internal defaults. "
        "COMPARE HYDROGRAPHS AT REACH A1009999_2147_R TO VERIFY."
    )
else:
    print("No Muskingum Cunge parameters found to update.")

## Step 5: Run Converted Project with HMS 4.11

In [ ]:
# Run with HMS 4.x
if hms_411_path and hms_411_path.exists():
    script = HmsJython.generate_compute_script(
        project_path=project_411,
        run_name=run_name,
        python2_compatible=False
    )
    
    print(f"Executing CONVERTED run with HMS {hms_411_path.name}...")
    result_411 = HmsJython.execute_script(
        script_content=script,
        hms_exe_path=hms_411_path,
        working_dir=project_411,
        timeout=300
    )
    
    print(f"\nConverted Run Success: {result_411[0]}")
    if "Computation completed" in result_411[1]:
        print(f"HMS {hms_411_path.name} converted run completed successfully.")
        modeling_log["converted_run"] = {
            "version": f"HMS {hms_411_path.name}",
            "run_name": run_name,
            "success": True,
            "dss_file": str(project_411 / "A1000000.dss")
        }
    else:
        print(f"Run may have failed. Check output.")
else:
    print("HMS 4.x not installed")

## Step 6: Extract and Compare Results

**Critical QAQC Step**: Compare outputs from both versions to verify equivalence.

In [ ]:
# Define DSS file paths
dss_33 = project_path / "A1000000.dss"
dss_411 = project_411 / "A1000000.dss"

print("DSS Output Files:")
print(f"  HMS 3.3 Baseline: {dss_33}")
print(f"    Exists: {dss_33.exists()}, Size: {dss_33.stat().st_size if dss_33.exists() else 'N/A':,} bytes")
print(f"  HMS 4.11 Converted: {dss_411}")
print(f"    Exists: {dss_411.exists()}, Size: {dss_411.stat().st_size if dss_411.exists() else 'N/A':,} bytes")

In [ ]:
# Results comparison checklist
print("\n" + "=" * 60)
print("RESULTS COMPARISON CHECKLIST")
print("=" * 60)
print("""
The following elements should be compared between HMS 3.3 and 4.11 outputs:

1. SUBBASINS (131 total)
   - Peak outflow (cfs)
   - Time to peak
   - Total runoff volume (ac-ft)
   - Precipitation depth (in)

2. REACHES (94 total, including A1009999_2147_R with Muskingum Cunge)
   - Peak outflow (cfs)
   - Peak inflow (cfs)
   - Time to peak
   - Attenuation (peak reduction)
   - Translation (lag time)

3. JUNCTIONS (multiple)
   - Peak combined flow (cfs)
   - Time to peak

4. OUTLET
   - Peak discharge (cfs)
   - Time to peak
   - Total volume (ac-ft)
   - Full hydrograph comparison

CRITICAL REACH FOR VERIFICATION:
  Reach A1009999_2147_R - Muskingum Cunge routing was modified
  Compare inflow/outflow hydrographs in detail
""")

### Automated Results Comparison

If DSS operations are available (requires pyjnius + Java), the following cell will automatically compare peak flows between the HMS 3.x baseline and HMS 4.x converted runs, flagging any elements with >1% difference.

In [ ]:
# Automated Results Comparison
# Compare peak flows between HMS 3.x baseline and HMS 4.x converted runs

import gc
from hms_commander import HmsResults, HmsDss

def read_peaks_safely(dss_file, label):
    """
    Safely read peak flows from DSS file with proper error handling.
    
    Handles PermissionError when DSS file is locked by another process
    (e.g., HMS GUI, HEC-DSSVue, or a previous notebook run).
    """
    try:
        peaks = HmsResults.get_peak_flows(dss_file)
        return peaks, None
    except PermissionError as e:
        error_msg = f"""
PermissionError accessing {label} DSS file: {dss_file}

LIKELY CAUSES:
  1. HEC-HMS GUI has the file open - Close HMS and try again
  2. HEC-DSSVue has the file open - Close DSSVue and try again  
  3. Previous notebook run didn't release file - Restart kernel
  4. Another Python process is using the file - Check Task Manager

SOLUTIONS:
  - Close any HEC applications that may have the DSS file open
  - Restart the Jupyter kernel (Kernel > Restart)
  - Wait a few seconds and re-run this cell
  - If problem persists, manually compare in HEC-DSSVue
"""
        return None, error_msg
    except Exception as e:
        return None, f"Error reading {label}: {e}"

# Check if DSS operations are available
if HmsDss.is_available():
    print("DSS operations available - performing automated comparison\n")
    
    comparison_results = {}
    
    # Check if both DSS files exist and were generated
    if dss_33.exists() and dss_411.exists():
        print("=" * 60)
        print("AUTOMATED PEAK FLOW COMPARISON")
        print("=" * 60)
        
        # Force garbage collection before accessing DSS files
        # This helps release any lingering file handles from previous operations
        gc.collect()
        
        # Read peaks with proper error handling
        peaks_33, error_33 = read_peaks_safely(dss_33, "HMS 3.x baseline")
        
        if error_33:
            print(error_33)
            log_warning(f"Could not read HMS 3.x DSS file: {dss_33}")
        else:
            # Force GC between file accesses
            gc.collect()
            
            peaks_411, error_411 = read_peaks_safely(dss_411, "HMS 4.x converted")
            
            if error_411:
                print(error_411)
                log_warning(f"Could not read HMS 4.x DSS file: {dss_411}")
            elif peaks_33 is not None and peaks_411 is not None:
                # Both files read successfully - perform comparison
                print(f"\nHMS 3.x ({hms_33_path.name if hms_33_path else '3.x'}) peak flows: {len(peaks_33)} elements")
                print(f"HMS 4.x ({hms_411_path.name if hms_411_path else '4.x'}) peak flows: {len(peaks_411)} elements")
                
                # Merge for comparison
                if not peaks_33.empty and not peaks_411.empty:
                    # Rename columns for clarity
                    peaks_33_renamed = peaks_33.rename(columns={'peak_flow': 'Peak_3x_cfs'})
                    peaks_411_renamed = peaks_411.rename(columns={'peak_flow': 'Peak_4x_cfs'})
                    
                    # Set element as index for merging
                    peaks_33_renamed = peaks_33_renamed.set_index('element')
                    peaks_411_renamed = peaks_411_renamed.set_index('element')
                    
                    # Merge on element name
                    comparison = pd.merge(
                        peaks_33_renamed[['Peak_3x_cfs']], 
                        peaks_411_renamed[['Peak_4x_cfs']], 
                        left_index=True, 
                        right_index=True,
                        how='outer'
                    )
                    
                    # Calculate differences
                    comparison['Diff_cfs'] = comparison['Peak_4x_cfs'] - comparison['Peak_3x_cfs']
                    comparison['Diff_pct'] = (comparison['Diff_cfs'] / comparison['Peak_3x_cfs'] * 100).round(2)
                    
                    # Flag elements with >1% difference
                    comparison['Flag'] = comparison['Diff_pct'].abs() > 1.0
                    
                    print("\n" + "-" * 60)
                    print("Peak Flow Comparison Summary:")
                    print("-" * 60)
                    print(f"Total elements compared: {len(comparison)}")
                    print(f"Elements with >1% difference: {comparison['Flag'].sum()}")
                    print(f"Max difference: {comparison['Diff_pct'].max():.2f}%")
                    print(f"Min difference: {comparison['Diff_pct'].min():.2f}%")
                    print(f"Mean difference: {comparison['Diff_pct'].mean():.2f}%")
                    
                    # Show flagged elements
                    flagged = comparison[comparison['Flag']]
                    if len(flagged) > 0:
                        print("\n" + "-" * 60)
                        print("FLAGGED ELEMENTS (>1% difference) - REQUIRES REVIEW:")
                        print("-" * 60)
                        print(flagged.to_string())
                    else:
                        print("\nNo elements exceed 1% difference threshold.")
                    
                    # Store in log
                    comparison_results = {
                        "total_elements": len(comparison),
                        "flagged_elements": int(comparison['Flag'].sum()),
                        "max_diff_pct": float(comparison['Diff_pct'].max()),
                        "min_diff_pct": float(comparison['Diff_pct'].min()),
                        "mean_diff_pct": float(comparison['Diff_pct'].mean()),
                        "flagged_list": flagged.index.tolist() if len(flagged) > 0 else []
                    }
                    modeling_log["results_comparison"] = comparison_results
                    
                    # Display top differences
                    print("\n" + "-" * 60)
                    print("Top 10 Largest Differences:")
                    print("-" * 60)
                    print(comparison.nlargest(10, 'Diff_pct', keep='first')[['Peak_3x_cfs', 'Peak_4x_cfs', 'Diff_cfs', 'Diff_pct']].to_string())
                else:
                    print("One or both DSS files returned empty results")
        
        # Force final garbage collection to release any file handles
        gc.collect()
                
    else:
        print("One or both DSS files not found - manual comparison required")
        if not dss_33.exists():
            print(f"  Missing: {dss_33}")
        if not dss_411.exists():
            print(f"  Missing: {dss_411}")
else:
    print("DSS operations not available (requires pyjnius + Java)")
    print("Manual comparison using HEC-DSSVue is required.")

## Step 7: Save Modeling Change Log

In [ ]:
# Add summary to log
modeling_log["summary"] = {
    "total_changes": len(modeling_log["changes"]),
    "total_warnings": len(modeling_log["warnings"]),
    "files_modified": list(set([c["file"] for c in modeling_log["changes"]])),
    "requires_engineer_review": len(modeling_log["warnings"]) > 0,
    "critical_elements": ["A1009999_2147_R (Muskingum Cunge reach)"]
}

# Save log to JSON
log_path = project_411 / "HMS_CONVERSION_LOG.json"
with open(log_path, 'w') as f:
    json.dump(modeling_log, f, indent=2)
print(f"Modeling log saved to: {log_path}")

In [ ]:
# Save human-readable version
log_txt_path = project_411 / "HMS_CONVERSION_LOG.txt"
with open(log_txt_path, 'w') as f:
    f.write("=" * 70 + "\n")
    f.write("HMS VERSION CONVERSION LOG\n")
    f.write("=" * 70 + "\n\n")
    
    f.write(f"Project: {modeling_log['project_name']}\n")
    f.write(f"Conversion Date: {modeling_log['conversion_date']}\n")
    f.write(f"Source Version: {modeling_log['source_version']}\n")
    f.write(f"Target Version: {modeling_log['target_version']}\n")
    f.write(f"Engineer Review Required: {modeling_log['summary']['requires_engineer_review']}\n\n")
    
    f.write("-" * 70 + "\n")
    f.write("CHANGES MADE\n")
    f.write("-" * 70 + "\n\n")
    
    for i, change in enumerate(modeling_log["changes"], 1):
        f.write(f"{i}. {change['category']}: {change['description']}\n")
        f.write(f"   File: {change['file']}\n")
        f.write(f"   Old: {change['old_value']}\n")
        f.write(f"   New: {change['new_value']}\n")
        f.write(f"   Expected Impact: {change['expected_impact']}\n\n")
    
    if modeling_log["warnings"]:
        f.write("-" * 70 + "\n")
        f.write("WARNINGS - REQUIRE ENGINEER REVIEW\n")
        f.write("-" * 70 + "\n\n")
        
        for i, warning in enumerate(modeling_log["warnings"], 1):
            f.write(f"{i}. {warning['message']}\n\n")
    
    f.write("\n" + "=" * 70 + "\n")
    f.write("QAQC VERIFICATION CHECKLIST\n")
    f.write("=" * 70 + "\n\n")
    f.write("[ ] Compare outlet hydrograph peak and timing\n")
    f.write("[ ] Compare reach A1009999_2147_R inflow/outflow\n")
    f.write("[ ] Verify all subbasin peaks within 1% tolerance\n")
    f.write("[ ] Verify all junction peaks within 1% tolerance\n")
    f.write("[ ] Review Muskingum Cunge Index Celerity value\n")
    f.write("[ ] Sign-off by reviewing engineer\n\n")
    f.write(f"Engineer Signature: _____________________  Date: __________\n")

print(f"Human-readable log saved to: {log_txt_path}")

## Summary: Conversion Complete

In [ ]:
print("=" * 70)
print("CONVERSION SUMMARY")
print("=" * 70)
print(f"\nTotal changes made: {len(modeling_log['changes'])}")
print(f"Warnings requiring review: {len(modeling_log['warnings'])}")
print(f"\nFiles modified:")
for f in modeling_log['summary']['files_modified']:
    print(f"  - {f}")

print(f"\nCritical elements for verification:")
for elem in modeling_log['summary']['critical_elements']:
    print(f"  - {elem}")

print("\n" + "-" * 70)
print("NEXT STEPS FOR ENGINEER:")
print("-" * 70)
print("""
1. Open both DSS files in HEC-DSSVue
2. Compare outlet hydrographs (peak, timing, volume)
3. Compare reach A1009999_2147_R routing (attenuation, translation)
4. Verify Index Celerity = 5 fps is appropriate for channel
5. Document any differences > 1% in the conversion log
6. Sign off on QAQC checklist
""")

print("\nOutput files:")
print(f"  Conversion log (JSON): {project_411 / 'HMS_CONVERSION_LOG.json'}")
print(f"  Conversion log (TXT):  {project_411 / 'HMS_CONVERSION_LOG.txt'}")

## Next Steps

- **08_m3_models.ipynb**: Discover and extract M3 model projects
- **07_execution_jython.ipynb**: Advanced Jython execution patterns
- **HCFCD M3 Models**: https://www.m3models.org/